In [1]:
import os

In [2]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [3]:
from config import spark

22/04/25 18:27:04 WARN Utils: Your hostname, eniac resolves to a loopback address: 127.0.1.1; using 192.168.100.6 instead (on interface wlp3s0)
22/04/25 18:27:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/25 18:27:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/25 18:27:04 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [4]:
datalake = '/media/fabio/19940C2755DB566F/PAMepi/datalake/DATALAKE'

In [5]:
version = '2022-04-19/'

In [6]:
df = spark.read.csv(os.path.join(datalake, version, 'google_mobility'), header=True)

In [7]:
df = df.where(F.col('country_region') == 'Brazil')

In [8]:
df = df.withColumn('sub_region_1', F.regexp_replace('sub_region_1', 'State of ', ''))

In [10]:
df = df.withColumnRenamed('sub_region_1', 'uf').withColumnRenamed('sub_region_2', 'mun_name')

In [12]:
columns = [
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline',
]

In [13]:
df.groupby('date', 'mun_name', 'uf') \
    .agg(*[F.count(c).alias(c) for c in columns]) \
    .filter((F.col('date') >= '2020-01-01') & (F.col('date') <= '2022-04-19')) \
    .orderBy('date') \
    .write.csv(os.path.join(datalake, version, 'preprocess', 'google_mobility'), header=True, mode='overwrite')